In [18]:
from fastai import *
from fastai.tabular import *
from fastai.tabular.all import *

In [11]:
import pandas as pd
import joblib

In [5]:
train_df = pd.read_csv('data/clean_train.csv')
test_df = pd.read_csv('data/clean_test.csv')

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
class PandasSimpleImputer(SimpleImputer):
    """A wrapper around `SimpleImputer` to return data frames with columns.
    """

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns=self.columns)

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
feature_extractor = joblib.load('feature_extractor.joblib')

In [13]:
from sklearn.model_selection import train_test_split
target = 'target'

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_df.drop(target, axis=1), 
    train_df[target], 
    test_size=0.05, 
    random_state=42
)

In [15]:
train_values = feature_extractor.transform(X_train).toarray()
valid_values = feature_extractor.transform(X_valid).toarray()
test_values = feature_extractor.transform(test_df).toarray()

In [47]:
train_df = pd.DataFrame(np.vstack((train_values, valid_values)))

In [48]:
train_df['target'] = y_train

In [94]:
train_df.fillna(0, inplace=True)

In [95]:
data = TabularDataLoaders.from_df(
    train_df, 
    procs=[Categorify, FillMissing, Normalize],
    cont_names=[i for i in range(train_values.shape[1])], 
    y_names="target",
    valid_idx=list(range(train_values.shape[0],train_df.shape[0])),
    bs=64
)

In [100]:
learn = tabular_learner(data, metrics=F1Score(average='weighted'))

In [101]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,f1_score,time
0,15.154168,849.337585,0.251096,00:19


epoch,train_loss,valid_loss,f1_score,time
0,14.974614,1761911.000000,0.251096,00:17
1,14.753739,1683235840.000000,0.251096,00:18
2,14.028121,5697649664.000000,0.251096,00:19


KeyboardInterrupt: 